### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle
from countpassenger import Dataset

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-14"
SAVE_DIR = os.path.join(
    conf.RESOURCES_PROCESSED_DIR, f"2024-04-28/count-passengers-{CURRENT_CAM}-vehicle.csv"
)
load_param = dict(
    vehicle_path_rel="2024-04-28/mbk-tourist-vehicle-object-20240428.csv",
    cross_path_rel="2024-04-28/mbk-tourist-raw-cross-object-20240428.csv",
    reverse_path_rel="2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv",
)

In [2]:
df_vehicle, df_cross, df_reverse = Dataset.load_dataset_from_paths(**load_param)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross/Reverse

In [3]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

In [4]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [7]:
df_reverse_with_reverse_count = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle_with_cross_and_reverse = match_cross_to_vehicle(
    df_cross,
    df_reverse_with_reverse_count,
)

# df_vehicle = df_vehicle[
#     (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
# ]
print(df_vehicle_with_cross_and_reverse[
    [
        *conf.VEHICLE_INTEREST_NON_SNAPSHOT,
        "cross_count",
        "reverse_count",
        # "frame_no",
    ]
].to_string())



      is_plate_valid  travel_distance  vehicle_type_confidence  lifetime  \
2328           False      1241.330036                 0.959638     1.760   
2319            True      1232.335681                 0.672324     2.999   
2320            True      1224.568804                 0.686204     1.640   
2321            True      1357.941952                 0.389375     4.479   
2189            True      2733.894467                 0.594424    97.092   
...              ...              ...                      ...       ...   
64             False      1210.871857                 0.920888     1.759   
61             False      1267.847536                 0.731769     1.760   
59              True      1205.600697                 0.896717     1.279   
31              True      1300.709908                 0.298450     4.119   
24             False      1242.783670                 0.377606     2.120   

      ymin plate_number  xmin  province_confidence similarized_plate_number  \
2328    

In [6]:
# df_vehicle_with_cross_and_reverse.to_csv(path_or_buf=SAVE_DIR, index=False)

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?